In [83]:
from addict import Dict
import argparse 
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
import torch
import pandas as pd
import numpy as np
parser = argparse.ArgumentParser()
parser.add_argument('-hyperparemeter','--top_k', type=int, default=5, help="")
parser.add_argument('--summary_dir_in', type=str, default="/mnt/External/Seagate/FedASR/LLaMa2/dacs/EmbFeats/Lexical/Embeddings/text_data2vec-audio-large-960h_Phych-anomia", help="")
args = parser.parse_args(args=[])
top_k=args.top_k

df_train = pd.read_pickle(f"{args.summary_dir_in}/train.pkl")
df_val = pd.read_pickle(f"{args.summary_dir_in}/dev.pkl")
df_test = pd.read_pickle(f"{args.summary_dir_in}/test.pkl")


class FindTopK_class:
    def __init__(self, method=1, top_n=5):
        self.method=method
        self.similarity='cosine_similarity'
        # self.similarity='distance'
        self.relatedness_fn=lambda x, y: cosine_similarity(x, y) if self.similarity=='cosine_similarity' else lambda x, y: 1 - spatial.distance.cosine(x, y)
        self.top_n=top_n
        if method==1:
            self.prompt_template = "{text}"
        elif method == 2:
            self.prompt_template = "This sentence: {text} means in one word:"
        
        self.df_dictionary=None
    def update_dictionary(self,df_train,selected_cols=['session','Psych_Summary','Embedding']):
        self.df_dictionary=df_train[selected_cols]
    def query(self, sentence_embeddings,top_n=None):
        if self.similarity=='cosine_similarity' :
            strings_and_relatednesses = [
                (row["session"], self.relatedness_fn(np.array(sentence_embeddings).reshape(1,-1), np.array(row["Embedding"]).reshape(1,-1)))
                for i, row in self.df_dictionary.iterrows()
            ]
        else:
            strings_and_relatednesses = [
                (row["session"], self.relatedness_fn(sentence_embeddings.squeeze(0), row["Embedding"].squeeze(0)))
                for i, row in self.df_dictionary.iterrows()
            ]

        strings, relatednesses = zip(*strings_and_relatednesses)

        Rank_df = pd.DataFrame({
            "session": self.df_dictionary["session"],
            "relatedness": relatednesses
        })
        if type(top_n)!=int:
            return Rank_df
        else:
            strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
            strings, relatednesses = zip(*strings_and_relatednesses)
            print("Top5: ",strings[:self.top_n])
            return strings[:self.top_n]
def ConstructCompareBase(df_train):
    Similar_finder=FindTopK_class()
    Similar_finder.update_dictionary(df_train)
    return Similar_finder

Similar_finder=ConstructCompareBase(df_train)

# df_data=df_train
# if 'Similarity_Emb' not in df_data.columns and 'Similarity_IDs' not in df_data.columns:
#     Similarity_Emb_dict={}
#     Similarity_IDs_dict={}
#     for session, row in df_data.iterrows():
#         top_relate=Similar_finder.query(row['Embedding'])
#         top_k_relate=top_relate.sort_values(by='relatedness', ascending=False).iloc[:top_k]
#         top_k_relate_indexes=list(top_k_relate.index)

#         average_Emb=np.mean(np.vstack(df_train.loc[top_k_relate_indexes,'Embedding']),axis=0)
#         topk_related_ID="/".join(top_k_relate_indexes)

#         # df_data.loc[i,'Similarity_Emb']=average_Emb
#         # df_data.loc[i,'Similarity_IDs']=topk_related_ID
#         Similarity_Emb_dict[session]=average_Emb
#         Similarity_IDs_dict[session]=topk_related_ID
#     df_data['Similarity_Emb'] = df_data.index.to_series().apply(lambda x: Similarity_Emb_dict.get(x, []))
#     df_data['Similarity_IDs'] = df_data.index.to_series().apply(lambda x: Similarity_IDs_dict.get(x, []))

def Extend_similarity_Emb(df_data, df_train, top_k=5):
    if 'Similarity_Emb' not in df_data.columns and 'Similarity_IDs' not in df_data.columns:
        Similarity_Emb_dict={}
        Similarity_IDs_dict={}
        for session, row in df_data.iterrows():
            top_relate = Similar_finder.query(row['Embedding'])
            top_k_relate = top_relate.sort_values(by='relatedness', ascending=False).iloc[:top_k]
            top_k_relate_indexes = list(top_k_relate.index)

            average_Emb = np.mean(np.vstack(df_train.loc[top_k_relate_indexes, 'Embedding']), axis=0)
            topk_related_ID = "/".join(top_k_relate_indexes)

            
            Similarity_Emb_dict[session]=average_Emb
            Similarity_IDs_dict[session]=topk_related_ID
        df_data['Similarity_Emb'] = df_data.index.to_series().apply(lambda x: Similarity_Emb_dict.get(x, []))
        df_data['Similarity_IDs'] = df_data.index.to_series().apply(lambda x: Similarity_IDs_dict.get(x, []))
    return df_data
df_train=Extend_similarity_Emb(df_train, df_train, top_k=5)
df_test=Extend_similarity_Emb(df_test, df_train, top_k=5)
df_val=Extend_similarity_Emb(df_val, df_train, top_k=5)

pd.to_pickle(df_train,f"{args.summary_dir_in}/train.pkl")
pd.to_pickle(df_val,f"{args.summary_dir_in}/dev.pkl")
pd.to_pickle(df_test,f"{args.summary_dir_in}/test.pkl")